In [ ]:
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template

# app.route("/SelectFile")
  -SelectFile.html

In [ ]:
#pybo.py
from flask import Flask
from flask import render_template
from waitress import save
app = Flask(__name__)

@app.route("/")
def hello_pybo():
  return 'Hello, S3'

# SelectFile 이라는 URL이 실행되면 아래의 함수 실행
@app.route("/SelectFile")
def select_file():
  # templates 폴더의 SelectFile.html실행
  return render_template("SelectFile.html")

save(app, host='0,0,0,0',port =8081)

In [ ]:
# SelectFile.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
    <form action="UploadFile" method="post" enctype="multipart/form-data">
        <input type="file" name="select"/><br/>
        <input type="submit" value="전송">
    </form>
</body>
</html>

# Flask aws 파일 업로드

- aws에서 IAM 설정후 csv 다운
- terminal에서 aws configure 입력

In [ ]:
!pip install boto3
!pip install awscli
import boto3
s3 = boto3.resource("s3")
for bucket in s3.buckets.all():
  print(bucket.name)
# 설정한 s3 bucket 이름 나옴

- 업로드 한 파일을 임시로 저장할 temporary 폴더 생성
- 같은 이름의 파일올라갈시 뒤에 번호 붙이기

In [ ]:
#pybo.py 에 추가
from flask import request
import boto3

# UploadFile URL이 실행될떄 아래 함수 실행
# methods = ["POST"] : SelectFile.html 페이지에서 전송버튼을 클릭했을떄만 실행하게
@app.route("/UploadFile", methods = ["POST"])
def upload_file():
  if request.methoud == "POST":

    file = request.files['select']

    # 선택한 파일이 존재하면
    if file.filename != '':
      # 선택한 파일이름을 save_file_name 변수에 저장
      save_file_name =file.filename

      count = 1


      while path.exists("./temporary/"+save_file_name):
        print(save_file_name+"이미 존재")

        

        # path.splitext(save_file_name)의 리턴값을 순서대로 file_name, ext 에 순서대로 저장
        # path.splitext(save_file_name) 리턴값이 {"blog","txt"}라면
        # file_name에는 "blog" , ext에는 "txt" 저장
        if len(path.splitext(save_file_name)) == 2:
          file_name,ext = path.splitext(save_file_name)

        else:
          # 확장자가 없는경우
          # file_name에 save_file_name 대입
          file_name = save_file_name
          ext = ""
        
        save_file_name = file_name+str(count) +ext
        count += 1

      # SelectFile.html에서 선택한 file을 temporary폴더에save_file_name 이름으로 임시저장
      file.save("./temporary/" + save_file_name)

      # aws의 s3에 접속
      s3 = boto3.client("s3")

      # aws s3에 파일업로드
      # 1번째 변수 : 로컬에서 올릴 파일이름
      # 2번째 변수 : s3 버킷이름
      # 3번쨰 변수 : 버킷에 저장될파일이름

      s3.upload_file("./temporary/"+ save_file_name,"s3d이름",save_file_name)
      return save_file_name
  return "s3에 업로드 실패"

# S3에 업로드한 파일 리스트 조회

In [ ]:
@app.route('./ViewFileList')
def view_file_list():
  # aws의 s3 접속
  s3 = boto3.client('s3')
  #버킷에서 object들을 가져오기 위해 list_objects_v2를 이용한다.
  paginartor = s3.get_paginator('list_objects_v2')
  # s3 버킷에 업로드한 파일의 정보를 리턴
  response_iterator = paginator.paginate(
                              Bucket = "버킷이름"
                                      )
  
  # s3에 저장된 파일들을 paginator.paginate로 불러오는데
  # 타입객체가 iterator 형식
  # 편하게 list형태로 전환
  file_list = list(response_iterator)

  # file_list[0] : 멘 처음 50개 정보 리턴
  # { "Contents" :  [파일1 ,파일2 ...]}
  return render_template("ViewFileList.html",
                         file_list = file_list[0]['Contents']
                         
                         )

In [ ]:
# ViewFileList.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>업로드 파일 리스트</title>
</head>
<body>
    # 위에서 render로 file_list 던짐
    # file_list 내용있으면 True , 없으면 False
    {% if file_list %}
        {% for file in file_list %}
            <p>
                <form action = "/DownloadFile" method="post">
                    파일명 : {{ file.Key }}
                    <input type="hidden"
                           name = "file_name"
                           value = "{{ file.Key }}"/>
                           # hidden: 화면에는 출력안하고, pybo로 데이터만 전송
                           # name :pybo로 데이터 전송시 사용할 이름
                           # value : pybo 로 전송할 데이터
                    <input type="submit" value ="다운로드"/>
                </form>
            </p>
        {% endfor %}
    {% else %}
        <p>파일이 없습니다</p>
    {% endif %}
</body>
</html>

# Flask aws 파일 다운로드

In [ ]:
from flask import Response
import urllib

In [ ]:
# methods =["POST"] : input type = "submit" 인 버튼을 클릭했을때 실행
@app.route("./DowunladFile", methods=['POST'])
def download_file():
  #f = request.files['file']
  #일반적으로 폼에서 데이터를 가져올 때와, 비슷한 방식이다. 
  #위 코드는 request 객체의 'file'이라는 이름의 폼으로 전송된 파일에 해당한다.
  file_nmae = request.form["file_name"]
  
  s3 = boto3.client("s3")

  # html에서받은 file_name의 값을 가지고
  # s3의 버킷에서 객체를 찾아 리턴해서 file에 저장
  file = s3.get_object(
                        Bucket = "버킷이름",
                        Key = file_name
                        )
  filename_header = 'filename*=UTF-8\'\'%s' %urllib.parse.quote(file_name.encode('utf-8'))


  # s3.get_object에서 찾은 객체를 ..read해서 내용을 가져옴
  # Response(웹브라우져로 내용전송) 
  return Response(
                file['Body'].read(),
                headers={"Content-Disposition" : "attachment;"+filename_header}
                  )

# 학생관리 프로그램


- 학생 전체 리스트


In [ ]:
from flask import Flask
from waitress import server
app = Flask(__name__)

@app.route("/")
def hello_student_manager():
  
  # boto3.resource("dynamodb") :
  # dynamodb에 접속해서 데이터 수정할수 있는 객체 리턴
  dynamodb = boto3.resource("dynamodb")

  # 위에서 받은 객체 'dynamodb'의 UnivStudent 테이블을
  # 수정할수있게 객체 리턴
  table =  dynamodb.Table("UnivStudent")
  
  # 위에서 리턴받은 테이블을 조회
  response = table.scan()

  return render_template("ViewStudentList.html",
                          student_list = response['Items']
                          )
serve(app, host='0.0.0.0', port =8082)

In [ ]:
# ViewStudentList.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>전체 학생 리스트</title>
</head>
<body>
{% if student_list %}
<table align="center" border="1">
    <tr>
        <td width="150">학번</td>
        <td width="150">이름</td>
        <td width="150">학과</td>
        <td width="150">평점</td>
        <td width="150">동아리</td>
    </tr>
    {% for student in student_list %}
    <tr>
        <td>{{ student.univ_id }}</td>
        <td>{{ student.univ_name }}</td>
        <td>{{ student.major }}</td>
        <td>{{ student.avg_credit }}</td>
        <td>{{ student.circle }}</td>
    </tr>
    {% endfor %}
</table>
{% else %}
<p>학생저보가 없습니다</p>
{% endif %}
</body>
</html>

- 학생 추가하기


In [ ]:
# 추가 버튼
@app.route('/AddStudentForm')
def add_student_fomr():
  return render_template("AddStudentForm.html")

In [ ]:
# AddStudentForm.html
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>학생정보 추가</title>
</head>
<body>
<h1 align="center">학생추가</h1>
<form action="AddStudent" method="post">
    학번:<input name="univ_id"/><br/>
    이름:<input name="univ_name"/><br/>
    학과:<input name="major"><br/>
    학점:<input name="avg_credit"><br/>
    동아리:<input name="circle"><br/>
    <input type ="submit" value ="추가"/>
</form>
</body>
</html>

# ViewStudentList.html 에 추가
<p align="center">
    <a href="AddStudentForm">학생 추가</a>
</p>

In [ ]:
import flask import render_template, request, redirect, url_for
from decimal import Decimal

@app.route("/AddStudent", methods = ["POST"])
def add_student():
  univ_id = request.form["univ_id"]
  univ_name = request.form["univ_name"]
  major = request.form["major"]
  circle = request.form["circle"]
  avg_credit = request.form["avg_credit"]

  dynamodb = boto3.resource("dynamodb")
  Table = dynamodb.Table('UnivStudent')
  
  if univ_id != "" and univ_name != "":
    new_student = {
              'univ_id' : Demical(univ_id),
              'univ_name' : univ_name
    }

    if major != "":
      new_student["major"] = major

    if circle != "":
      new_student["circle"] = circle
    
    if avg_credit != "":
      new_student["avg_credit"] =  Decimal(avg_credit)

    # 실제로 UnivStudent 테이블에 new_student 추가부분
    table.put_item(Item = new_student)
  
  return redirect(url_for("hello_student_manager"))



- 학생 삭제하기

In [ ]:
# ViewStudent.html에 추가
<td width="150">삭제</td>
################################
# 학생정보 삭제 버튼
{% for student in student_list %}
        <td>
            <form action="RemoveStudent" method="post">
                <input type="hidden" name="univ_id" value="{{ student.univ_id }}"/>
                <input type="hidden" name="univ_name" value="{{ student.univ_name }}"/>
                <input type = "submit" value="삭제">
            </form>
        </td>

In [ ]:
@app.route("/RemoveStudent", methods = ["POST"])
def remove_student():
  univ_name =  request.form["univ_name"]
  univ_id = request.form["univ_id"]

  dynamodb = boto3.resource["dynamodb"]
  table = dynamodb.Table("UnivStudent")

  will_remove_student = {
      "univ_name" : univ_name,
      "univ_id" : Decimal(univ_id)
  }

  table.delete_item(
      Key = will_remove_student
  )
  return redirect(url_for("hello_student_manager"))

- 학생 수정하기


In [ ]:
# ViewStudentList.html에 추가
<td width="150">수정</td>
<td>
            <form action="EditStudentForm" method="post">
                <input type="hidden" name="univ_id" value="{{ student.univ_id }}"/>
                <input type="hidden" name="univ_name" value="{{ student.univ_name }}"/>
                <input type = "submit" value="수정">
            </form>
</td>

In [ ]:
# 학생수정 화면
@app.route('/EditStudentForm', methods =["GET","POST"])
def edit_student_form():
  univ_id =  request.form["univ_id"]
  univ_name = request.fomr["univ_name"]

  dynamodb =  boto3.resource("dynamodb")
  table = dynamodb.Table("UnivStudent")

  will_edit_student = {
      "univ_name" : univ_name
      "univ_id" : Decimal(univ_id)
  }

  response =  table.get_item(Key = will_edit_student)
  
  return render_template("EditStudentForm.html",student= response["Item"])

In [ ]:
# EditStudentForm.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>학생정보 수정</title>
</head>
<body>
        <form action = "EditStudentForm" method="post">
            학번 : <input name ="univ_id" value="{{ student.univ_id}}" readonly/><br/>
            이름 : <input name ="univ_name" value="{{ student.univ_name}}" readonly/><br/>
            학과 : <input name ="major" value="{{ student.major}}" /><br/>
            학점 : <input name ="avg_credit" value="{{ student.avg_credit }}" /><br/>
            동아리 : <input name ="circle" value="{{ student.circle }}" /><br/>
            <input type = "submit" value="수정">
        </form>
</body>
</html>

In [ ]:
# 학생 수정 처리
@app.route("/EditStudent", methods=["GET","POST"])
def edit_student():
  univ_id =  request.form["univ_id"]
  univ_name = request.form["univ_name"]
  major = request.form["major"]
  circle =  request.form["circle"]
  avg_credit = request.form["avg_credit"]

  dynamodb = boto3.resource("dynamodb")
  table = dynamodb.Table("UnivStudent")

  if univ_id != "" and univ_name != "":
    edit_student{
        "univ_id" : Decimal(univ_id),
        "univ_name" : univ_name
    }

    if major != "":
      edit_student["major"] = major
    
    if circle != "":
      edit_student["circle"] =  circle

    if avg_credit != "":
      edit_student["avg_credit"] = avg_credit

    table.put_item(Item = edit_Student)
  
  return redirect(url_for("hello_student_manager"))